# Local Control Agent

The `LocalControlAgent` is a key component in the multi-agent system (MAS). It acts as a wrapper for a specific control algorithm (like a PID controller), enabling it to operate within the broader system by handling communication through the `MessageBus`. Its primary role is to listen for sensor data, process it using its encapsulated controller, and publish a control action.

## Core Concepts

- **Encapsulation:** It hides the complexity of the control logic inside a dedicated `Controller` object.
- **Event-Driven:** The agent doesn't run in a continuous loop. Instead, it reacts to messages. Its core logic is triggered when a new message arrives on its subscribed `observation_topic`.
- **Communication:** It uses a `MessageBus` to subscribe to topics (for sensor data and commands) and publish to topics (for control actions).

## Key Parameters

- `agent_id`: A unique identifier for the agent.
- `controller`: An instance of a class that implements the `Controller` interface.
- `message_bus`: The shared message bus for the system.
- `observation_topic`: The topic the agent listens to for sensor readings.
- `observation_key`: The specific key within the observation message that contains the process variable (e.g., 'water_level').
- `action_topic`: The topic where the agent publishes its calculated control signal.

## Simulation Example

The following example demonstrates how to set up and use a `LocalControlAgent`. We create a mock controller, a message bus, and the agent itself. We then simulate a sensor publishing a new measurement, and we'll see how the agent processes this information and publishes a corresponding control action, which is received by a mock actuator.

**Note:** To run this notebook, you need to be running the Jupyter server from the root directory of the project.

In [ ]:
import time
from swp.core.interfaces import Agent, Controller, State
from swp.central_coordination.collaboration.message_bus import MessageBus, Message
from swp.local_agents.control.local_control_agent import LocalControlAgent

# 1. Mock Controller for demonstration
class MockProportionalController(Controller):
    def __init__(self, gain):
        self.gain = gain
        self.setpoint = 5.0 # Target value
        
    def compute_control_action(self, state: State, dt: float) -> any:
        error = self.setpoint - state['process_variable']
        return self.gain * error

    def set_setpoint(self, new_setpoint: float):
        self.setpoint = new_setpoint

# 2. Setup the communication bus
bus = MessageBus()

# 3. Mock Actuator to receive control signals
received_actions = []
def actuator_callback(message: Message):
    print(f"Actuator received: {message}")
    received_actions.append(message)

bus.subscribe('gate_control_topic', actuator_callback)

# 4. Instantiate the Controller and the Agent
p_controller = MockProportionalController(gain=0.5)
control_agent = LocalControlAgent(
    agent_id="gate_controller_1",
    controller=p_controller,
    message_bus=bus,
    observation_topic='reservoir_level_topic',
    observation_key='water_level',
    action_topic='gate_control_topic',
    dt=1.0
)

# 5. Simulate a sensor publishing data
print("--- Sensor Publishing Data ---")
sensor_data = {'water_level': 4.8, 'timestamp': time.time()}
bus.publish('reservoir_level_topic', sensor_data)

# The message bus processes messages immediately
print("--- Verification ---")
assert len(received_actions) == 1
# Corrected assertion for floating point comparison
assert abs(received_actions[0]['control_signal'] - (0.5 * (5.0 - 4.8))) < 1e-9
print("Successfully received the correct control signal.")